In [1]:
%pip install --upgrade bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 20.2 MB/s eta 0:00:00:00:0100:01


In [2]:
import random
from datasets import load_dataset
import torch
from torch.utils.data import Dataset

class NoisyInstructDataset(Dataset):
    """
    Dataset for Noisy Instruction Following.
    Loads UltraFeedback and injects noise by flipping 'chosen' and 'rejected' responses.
    """
    def __init__(self, tokenizer, split='train', max_length=512, noise_ratio=0.3, seed=42):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.noise_ratio = noise_ratio
        
        random.seed(seed)
        
        # Load UltraFeedback (binarized preferences)
        # Using a smaller subset or a compatible version if needed. 
        # 'HuggingFaceH4/ultrafeedback_binarized' is a good pre-processed version.
        print(f"Loading dataset: HuggingFaceH4/ultrafeedback_binarized [{split}]...")
        try:
            self.dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=f"{split}_sft[:10000]") # Limit to 10k for speed as per plan
        except Exception as e:
            print(f"Error loading dataset: {e}. Fallback to local mock if needed (not implemented).")
            raise e
            
        self.data = []
        self.process_data()

    def process_data(self):
        print(f"Processing {len(self.dataset)} samples with {self.noise_ratio*100}% noise...")
        
        for i, item in enumerate(self.dataset):
            # Format: messages usually contain list of dicts [{'role': 'user', ...}, {'role': 'assistant', ...}]
            # In binarized version, we often have 'chosen' and 'rejected' lists of messages.
            
            prompt = item['prompt']
            chosen = item['chosen'][-1]['content'] # Last message is assistant response
            rejected = item['rejected'][-1]['content']
            
            is_noisy = False
            
            # Noise Injection
            if random.random() < self.noise_ratio:
                is_noisy = True
                # Flip: The "target" becomes the bad response
                target_response = rejected
            else:
                target_response = chosen
                
            # Construct input text for Causal LM
            # Format: "### Instruction: ... ### Response: ..." or ChatML
            # Phi-2 is flexible, let's use a simple format.
            text = f"Instruct: {prompt}\nOutput: {target_response}"
            
            self.data.append({
                'text': text,
                'is_noisy': is_noisy
            })
            
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        
        # Tokenize
        encodings = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        
        # Labels are same as input (standard Causal LM training)
        # We could mask the instruction part, but for simple benchmark, full sequence loss is fine.
        input_ids = encodings['input_ids'].squeeze()
        attention_mask = encodings['attention_mask'].squeeze()
        labels = input_ids.clone()
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
            'is_noisy': torch.tensor(item['is_noisy'], dtype=torch.long)
        }

In [3]:
"""
CASMO: Confident Adaptive Selective Momentum Optimizer

A production-ready PyTorch optimizer that extends Adam with confidence-based learning rate scaling.

Core Innovation: AGAR (Adaptive Gradient Alignment Ratio)
    AGAR = ||E[g]||² / (||E[g]||² + Var[g])
    
    Measures signal (consistent gradient direction) vs noise (random fluctuations).
    Naturally ranges from 0 (pure noise) to 1 (pure signal) for interpretable confidence metrics.

Performance:
    - Faster than AdamW on large models (-2% overhead with per-group mode)
    - Configurable granularity for speed/precision tradeoff
    - Pre-allocated buffers eliminate allocation overhead

Usage Example:
    >>> from casmo import CASMO
    >>> optimizer = CASMO(model.parameters(), lr=1e-3, weight_decay=0.01)
    >>> for epoch in range(num_epochs):
    ...     for batch in dataloader:
    ...         loss = model(batch)
    ...         loss.backward()
    ...         optimizer.step()
    ...         optimizer.zero_grad()

Reference: 
    Kingma & Ba (2015). "Adam: A Method for Stochastic Optimization"
    https://arxiv.org/abs/1412.6980
"""

from typing import Tuple, Optional, Callable, Dict, Any
import torch
import numpy as np
from collections import deque
import logging


class DDEAdapter:
    """
    Drift-Detecting EMA adapter for tau threshold adjustment.
    
    Tracks AGAR variance to adaptively adjust tau while preventing
    runaway adaptation to noise or memorization signals.
    O(1) memory and compute per step.
    """
    
    # EMA update rates
    EMA_MEAN_RATE = 0.001
    EMA_VAR_DECAY = 0.99
    EMA_VAR_RATE = 0.01
    
    # Adaptive gain bounds
    MIN_GAIN = 0.001
    MAX_GAIN = 0.01
    GAIN_SCALE = 0.1
    
    # Memorization detection threshold
    MEMORIZATION_FACTOR = 1.2
    
    def __init__(self, tau_init: float, tau_clip_range: Tuple[float, float], 
                 dead_zone_factor: float = 0.2):
        """
        Initialize the DDE adapter.
        
        Args:
            tau_init: Initial tau value
            tau_clip_range: (min, max) bounds for tau
            dead_zone_factor: Ignore deviations smaller than this fraction of tau.
                Prevents chasing noise. Default: 0.2 (20%)
        """
        self.tau = tau_init
        self.tau_calibrated: Optional[float] = None
        self.clip_range = tau_clip_range
        self.dead_zone = dead_zone_factor
        
        # EMA state for variance tracking
        self.mean_agar = tau_init
        self.ema_var = 0.01
    
    def update(self, agar_value: float) -> float:
        """
        Update tau threshold using variance-adaptive gain and dead zone filtering.
        
        Args:
            agar_value: Current AGAR measurement
            
        Returns:
            Updated tau value (clipped to valid range)
        """
        # Update EMA mean
        diff = agar_value - self.mean_agar
        self.mean_agar += self.EMA_MEAN_RATE * diff
        
        # Update EMA variance: Var[X] = E[(X - μ)²]
        self.ema_var = self.EMA_VAR_DECAY * self.ema_var + self.EMA_VAR_RATE * (diff ** 2)
        
        # Relative variance (scale-invariant)
        rel_var = self.ema_var / (self.mean_agar + 1e-8)
        
        # Prevent tau from chasing memorization signals
        if self.tau_calibrated is not None and agar_value > self.MEMORIZATION_FACTOR * self.tau_calibrated:
            # AGAR suspiciously high - likely overfitting, freeze tau
            return self.tau
        
        # Dead zone: only adapt if deviation exceeds threshold
        dead_zone_reference = self.tau_calibrated if self.tau_calibrated is not None else self.tau
        deviation = abs(agar_value - self.tau)
        if deviation > self.dead_zone * dead_zone_reference:
            # Variance-adaptive gain: higher variance → faster adaptation
            alpha = self.MIN_GAIN + min(rel_var * self.GAIN_SCALE, self.MAX_GAIN - self.MIN_GAIN)
            new_tau = (1 - alpha) * self.tau + alpha * agar_value
            
            # Never decrease tau below calibrated baseline
            if self.tau_calibrated is not None:
                new_tau = max(new_tau, self.tau_calibrated)
            
            self.tau = new_tau
        
        return float(np.clip(self.tau, self.clip_range[0], self.clip_range[1]))


class CASMO(torch.optim.Optimizer):
    """
    Confident Adaptive Selective Momentum Optimizer.
    
    Extends Adam with confidence-based learning rate scaling using AGAR metrics.
    Automatically adapts to gradient signal-to-noise ratio for improved convergence.
    
    Uses universal sigmoid-based confidence mapping that adapts to any noise distribution:
    - Clean data: High confidence baseline
    - Pervasive noise: Adaptive scaling with high c_min
    - Mixed batches: Strong discrimination via distribution statistics
    
    Args:
        params (iterable): Iterable of parameters to optimize or dicts defining parameter groups
        lr (float, optional): Learning rate. Default: 1e-3
        betas (Tuple[float, float], optional): Coefficients for computing running averages 
            of gradient and its square (β₁, β₂). Default: (0.9, 0.999)
        eps (float, optional): Term added to denominator for numerical stability. Default: 1e-8
        weight_decay (float, optional): Decoupled weight decay coefficient (AdamW-style). 
            Default: 0.0
        tau_init_steps (int, optional): Number of initial steps to collect AGAR samples 
            for automatic threshold calibration. Must be >= 50. Default: 500
            Recommended: max(500, int(0.05 * total_steps))
        tau_clip_range (Tuple[float, float], optional): Min/max bounds for tau threshold. 
            Default: (0.01, 0.5)
        tau_dead_zone (float, optional): Dead zone factor for tau adaptation.
            Ignores AGAR deviations smaller than this fraction of tau to prevent chasing noise.
            Default: 0.2 (20%)
        c_min (float, optional): Minimum confidence scaling factor to prevent learning rate 
            from becoming too small. Must be in [0, 1]. Default: 0.1
            Note: After calibration, c_min is automatically computed based on noise level.
        granularity (str, optional): AGAR computation granularity.
            - 'parameter': Per-parameter confidence scaling (~13% overhead on large models).
              Use for small models (<10M params) or when layer-specific adaptation matters.
            - 'group': Per-group confidence scaling (faster than AdamW on large models).
              Recommended for production use, large models (>10M params), and hyperparameter sweeps.
            Default: 'group'
        agar_clamp_factor (float, optional): Outlier clamping factor for AGAR computation.
            Clamps moment estimates to ±(mean * factor) to handle extreme values.
            Set to None to disable clamping. Default: 10.0
        log_level (int, optional): Logging verbosity. 0=silent, 1=errors, 2=warnings, 
            3=info. Default: 1
    
    Raises:
        ValueError: If any parameter is outside its valid range
        RuntimeError: If NaN or Inf gradients are detected during optimization
        NotImplementedError: If sparse gradients are encountered
    
    Note:
        This optimizer does not support sparse gradients. Use torch.optim.SparseAdam
        for sparse gradient scenarios.
    
    Example:
        >>> model = YourModel()
        >>> optimizer = CASMO(model.parameters(), lr=1e-3, weight_decay=0.01)
        >>> 
        >>> for epoch in range(num_epochs):
        ...     for batch in dataloader:
        ...         optimizer.zero_grad()
        ...         loss = model(batch)
        ...         loss.backward()
        ...         optimizer.step()
    """
    
    def __init__(
        self,
        params,
        lr: float = 1e-3,
        betas: Tuple[float, float] = (0.9, 0.999),
        eps: float = 1e-8,
        weight_decay: float = 0.0,
        tau_init_steps: int = 500,
        tau_clip_range: Tuple[float, float] = (0.01, 0.5),
        tau_dead_zone: float = 0.2,  # Large dead zone to prevent chasing memorization
        c_min: float = 0.1,
        granularity: str = 'group',
        agar_clamp_factor: Optional[float] = 10.0,
        log_level: int = 1,
    ):
        if not 0.0 <= lr:
            raise ValueError(f"Invalid learning rate: {lr}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon: {eps}")
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError(f"Invalid beta1: {betas[0]}")
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError(f"Invalid beta2: {betas[1]}")
        if not 0.0 <= weight_decay:
            raise ValueError(f"Invalid weight_decay: {weight_decay}")
        if not 0.0 <= c_min <= 1.0:
            raise ValueError(f"Invalid c_min: {c_min}")
        if tau_init_steps < 50:
            raise ValueError(f"tau_init_steps too small: {tau_init_steps} (minimum: 50)")
        if not 0.0 <= tau_dead_zone <= 1.0:
            raise ValueError(f"Invalid tau_dead_zone: {tau_dead_zone} (must be in [0, 1])")
        if granularity not in ['parameter', 'group']:
            raise ValueError(f"Invalid granularity: {granularity} (must be 'parameter' or 'group')")
        
        defaults = dict(
            lr=lr,
            betas=betas,
            eps=eps,
            weight_decay=weight_decay,
            tau_init_steps=tau_init_steps,
            tau_clip_range=tau_clip_range,
            tau_dead_zone=tau_dead_zone,
            c_min=c_min,
            granularity=granularity,
            agar_clamp_factor=agar_clamp_factor,
        )
        
        super().__init__(params, defaults)
        
        # Setup logging
        self.logger = logging.getLogger('CASMO')
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            handler.setFormatter(logging.Formatter('[CASMO] %(message)s'))
            self.logger.addHandler(handler)
        self.logger.setLevel(self._get_log_level(log_level))
        
        self._step_count = 0
        
        # Initialize per-group state for tau calibration and buffer reuse
        self._group_states: Dict[int, Dict[str, Any]] = {}
        for idx, group in enumerate(self.param_groups):
            group_tau_dead_zone = group.get('tau_dead_zone', tau_dead_zone)
            group_tau_clip_range = group.get('tau_clip_range', tau_clip_range)
            group_tau_init_steps = group.get('tau_init_steps', tau_init_steps)
            
            self._group_states[idx] = {
                'tau_adapter': DDEAdapter(1.0, group_tau_clip_range, dead_zone_factor=group_tau_dead_zone),
                'tau_initialized': False,
                'agar_buffer': deque(maxlen=group_tau_init_steps),
                'reuse_buffer_exp_avg': None,
                'reuse_buffer_exp_avg_sq': None,
                'current_agar': None,
                'current_confidence': None,
                'agar_mean': None,
                'agar_std': None,
                'agar_median': None,
                'agar_p10': None,
                'agar_p90': None,
                'c_min': c_min,
            }
    
    def _get_log_level(self, level: int) -> int:
        """
        Convert custom log level to Python logging level.
        
        Args:
            level: Custom level (0=silent, 1=error, 2=warning, 3=info)
            
        Returns:
            Python logging level constant
        """
        level_map = {
            0: logging.CRITICAL + 1,  # Silent
            1: logging.ERROR,
            2: logging.WARNING,
            3: logging.INFO,
        }
        return level_map.get(level, logging.WARNING)
    
    def _log(self, level: int, message: str) -> None:
        """
        Internal logging utility using Python logging module.
        
        Args:
            level: Message severity level (1=error, 2=warning, 3=info)
            message: Log message to output
        """
        if level == 1:
            self.logger.error(message)
        elif level == 2:
            self.logger.warning(message)
        elif level == 3:
            self.logger.info(message)
    
    def _validate_gradient(self, grad: torch.Tensor, group_idx: int) -> None:
        """
        Validate gradient for NaN, Inf, and sparse tensors.
        
        Args:
            grad: Gradient tensor to validate
            group_idx: Parameter group index for error messages
            
        Raises:
            RuntimeError: If NaN or Inf detected
            NotImplementedError: If sparse gradient detected
        """
        if torch.isnan(grad).any():
            raise RuntimeError(
                f"NaN gradient detected in parameter group {group_idx}. "
                "Consider using gradient clipping: torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)"
            )
        if torch.isinf(grad).any():
            raise RuntimeError(
                f"Inf gradient detected in parameter group {group_idx}. "
                "Check for numerical overflow in loss computation or model outputs."
            )
        if grad.is_sparse:
            raise NotImplementedError(
                "CASMO does not support sparse gradients. "
                "Use torch.optim.SparseAdam for sparse gradient scenarios, "
                "or convert gradients to dense format with grad.to_dense()."
            )
    
    def _init_param_state(self, p: torch.Tensor) -> Dict[str, Any]:
        """
        Initialize optimizer state for a parameter.
        
        Args:
            p: Parameter tensor
            
        Returns:
            Initialized state dictionary with step counter and moment estimates
        """
        state = self.state[p]
        if len(state) == 0:
            state['step'] = 0
            state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
            state['exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
        return state
    
    def _update_moments(self, state: Dict[str, Any], grad: torch.Tensor, beta1: float, beta2: float) -> None:
        """
        Update exponential moving averages of gradient moments.
        
        Args:
            state: Parameter state dictionary
            grad: Current gradient
            beta1: First moment decay rate (β₁)
            beta2: Second moment decay rate (β₂)
        """
        exp_avg = state['exp_avg']
        exp_avg_sq = state['exp_avg_sq']
        state['step'] += 1
        
        exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
        exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)
    
    def _apply_weight_update(self, p: torch.Tensor, state: Dict[str, Any], lr: float, 
                            weight_decay: float, eps: float, confidence: torch.Tensor,
                            beta1: float, beta2: float) -> None:
        """
        Apply Adam-style parameter update with confidence-scaled learning rate.
        
        Implements decoupled weight decay (AdamW) with bias-corrected moments
        and confidence-based learning rate modulation.
        
        Args:
            p: Parameter tensor to update
            state: Parameter state dictionary containing moments
            lr: Base learning rate
            weight_decay: Decoupled weight decay coefficient
            eps: Numerical stability constant (ε)
            confidence: Confidence scaling factor in [c_min, 1.0]
            beta1: First moment decay rate (β₁)
            beta2: Second moment decay rate (β₂)
        """
        exp_avg = state['exp_avg']
        exp_avg_sq = state['exp_avg_sq']
        step = state['step']
        
        # Bias correction
        bias_correction1 = 1 - beta1 ** step
        bias_correction2 = 1 - beta2 ** step
        m_hat = exp_avg / bias_correction1
        v_hat = exp_avg_sq / bias_correction2
        
        # Weight decay (decoupled)
        if weight_decay != 0:
            p.mul_(1 - lr * weight_decay)
        
        # Apply update with confidence-scaled learning rate
        denom = v_hat.sqrt().add_(eps)
        step_size = lr * confidence
        p.addcdiv_(m_hat, denom, value=-step_size)
    
    def _compute_agar(
        self,
        exp_avg: torch.Tensor,
        exp_avg_sq: torch.Tensor,
        eps: float,
        clamp_factor: Optional[float],
    ) -> torch.Tensor:
        """
        Compute Adaptive Gradient Alignment Ratio (AGAR) from exponential moving averages.
        
        AGAR quantifies the signal-to-noise ratio in gradients:
            AGAR = mean(signal / (signal + noise))
            where signal = (E[g])² (squared mean gradient per element)
                  noise = Var[g] = E[g²] - (E[g])² (gradient variance per element)
        
        Args:
            exp_avg (torch.Tensor): Exponential moving average of gradients (first moment)
            exp_avg_sq (torch.Tensor): Exponential moving average of squared gradients (second moment)
            eps (float): Small constant for numerical stability
            clamp_factor (Optional[float]): Outlier clamping factor (None to disable)
        
        Returns:
            torch.Tensor: Scalar AGAR value in range [0, 1], where:
                - 0 indicates pure noise (no consistent gradient direction)
                - 1 indicates pure signal (perfectly consistent gradients)
        
        Note:
            AGAR is computed per-element then uniformly averaged across all elements.
            This provides robustness across parameters with different scales.
            Uses raw moments to preserve the variance relationship Var[g] = E[g²] - (E[g])².
            Bias correction would distort this relationship and cause AGAR instability.
        """
        # Outlier protection: clamp extreme values based on gradient statistics
        if clamp_factor is not None:
            m_scale = exp_avg.abs().mean() + eps
            v_scale = exp_avg_sq.mean() + eps
            m_clamped = torch.clamp(exp_avg, min=-m_scale * clamp_factor, max=m_scale * clamp_factor)
            v_clamped = torch.clamp(exp_avg_sq, min=0.0, max=v_scale * clamp_factor)
        else:
            m_clamped = exp_avg
            v_clamped = exp_avg_sq
        
        # Signal: squared norm of mean gradient (consistent direction)
        signal_per_elem = m_clamped.pow(2)
        
        # Noise: gradient variance = E[g²] - (E[g])²
        noise_per_elem = torch.clamp(v_clamped - signal_per_elem, min=eps)
        
        # Compute mean AGAR across all elements (uniform weighting)
        agar_per_elem = signal_per_elem / (signal_per_elem + noise_per_elem + eps)
        agar = agar_per_elem.mean()
        
        return torch.clamp(agar, min=0.0, max=1.0)
    
    # Calibration constants
    MIN_CALIBRATION_SAMPLES = 50
    MIN_STD_THRESHOLD = 0.01  # Prevent division by zero
    
    # Coefficient of variation thresholds for adaptive c_min
    CV_HIGH_THRESHOLD = 0.5  # Bimodal distribution
    CV_MEDIUM_THRESHOLD = 0.3  # Some separation
    
    # Adaptive c_min values
    C_MIN_HIGH_VARIANCE = 0.1  # Strong discrimination for bimodal
    C_MIN_MEDIUM_VARIANCE = 0.3  # Moderate discrimination
    C_MIN_LOW_VARIANCE = 0.5  # High baseline for unimodal/pervasive noise
    
    def _calibrate_tau(self, agar_buffer: deque, tau_clip_range: Tuple[float, float], group_idx: int) -> float:
        """
        Universal tau calibration using distribution statistics.
        
        Computes distribution parameters for confidence mapping:
        - μ (mean): Central tendency of AGAR distribution
        - σ (std): Spread of AGAR distribution
        - p50 (median): Robust center estimate
        - p10, p90: Distribution bounds for outlier detection
        
        This approach works universally for:
        - Clean data: High μ, low σ → High confidence baseline
        - Pervasive noise: Low μ, low σ → Adaptive confidence scaling
        - Mixed batches: Medium μ, high σ → Bimodal confidence distribution
        
        Mathematical foundation:
            confidence(agar) = c_min + (1 - c_min) * sigmoid((agar - μ) / σ)
        
        This sigmoid mapping naturally adapts to any distribution shape.
        
        Args:
            agar_buffer: Collection of AGAR samples from initial training steps
            tau_clip_range: Safety bounds for tau (min, max)
            group_idx: Parameter group index for storing calibration results
        
        Returns:
            Calibrated tau threshold (median for robustness)
        """
        if len(agar_buffer) < self.MIN_CALIBRATION_SAMPLES:
            return tau_clip_range[1]
        
        samples = np.array(agar_buffer)
        
        # Distribution statistics
        mu = np.mean(samples)
        sigma = np.std(samples)
        median = np.median(samples)
        p10 = np.percentile(samples, 10)
        p90 = np.percentile(samples, 90)
        
        # Store distribution parameters for confidence mapping
        group_state = self._group_states[group_idx]
        group_state['agar_mean'] = float(mu)
        group_state['agar_std'] = float(max(sigma, self.MIN_STD_THRESHOLD))
        group_state['agar_median'] = float(median)
        group_state['agar_p10'] = float(p10)
        group_state['agar_p90'] = float(p90)
        
        # Adaptive c_min based on coefficient of variation (CV = σ/μ)
        # High CV → Lower c_min (strong discrimination for bimodal distributions)
        # Low CV → Higher c_min (prevent over-suppression for unimodal/pervasive noise)
        cv = sigma / (mu + 1e-8)
        if cv > self.CV_HIGH_THRESHOLD:
            c_min_adaptive = self.C_MIN_HIGH_VARIANCE
        elif cv > self.CV_MEDIUM_THRESHOLD:
            c_min_adaptive = self.C_MIN_MEDIUM_VARIANCE
        else:
            c_min_adaptive = self.C_MIN_LOW_VARIANCE
        
        group_state['c_min'] = float(c_min_adaptive)
        
        self._log(2, f"Calibrated AGAR distribution: μ={mu:.4f}, σ={sigma:.4f}, "
                     f"median={median:.4f}, CV={cv:.4f}, c_min={c_min_adaptive:.2f}")
        
        # Return median as tau (robust to outliers)
        return float(np.clip(median, tau_clip_range[0], tau_clip_range[1]))
    
    @torch.no_grad()
    def step(self, closure: Optional[Callable] = None) -> Optional[float]:
        """
        Perform a single optimization step.
        
        Args:
            closure (callable, optional): A closure that reevaluates the model and returns
                the loss. Optional for most optimizers but required for some (e.g., LBFGS).
        
        Returns:
            Optional[float]: Loss value if closure is provided, None otherwise
        
        Raises:
            RuntimeError: If NaN or Inf gradients are detected
            NotImplementedError: If sparse gradients are encountered
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()
        
        self._step_count += 1
        
        for group_idx, group in enumerate(self.param_groups):
            beta1, beta2 = group['betas']
            lr = group['lr']
            eps = group['eps']
            weight_decay = group['weight_decay']
            c_min = group['c_min']
            tau_init_steps = group['tau_init_steps']
            tau_clip_range = group['tau_clip_range']
            granularity = group['granularity']
            agar_clamp_factor = group['agar_clamp_factor']
            
            group_state = self._group_states[group_idx]
            
            # Per-group AGAR mode: compute once for all parameters in group
            if granularity == 'group':
                # Skip group if no parameters have gradients
                valid_params = [p for p in group['params'] if p.grad is not None]
                if not valid_params:
                    continue
                
                all_exp_avg = []
                all_exp_avg_sq = []
                
                # First pass: update momentum and collect states
                for p in group['params']:
                    if p.grad is None:
                        continue
                    
                    self._validate_gradient(p.grad, group_idx)
                    state = self._init_param_state(p)
                    self._update_moments(state, p.grad, beta1, beta2)
                    
                    all_exp_avg.append(state['exp_avg'].flatten())
                    all_exp_avg_sq.append(state['exp_avg_sq'].flatten())
                
                # Compute group-level AGAR using pre-allocated buffers
                if all_exp_avg:
                    # Allocate buffers on first use (amortized across all steps)
                    if group_state['reuse_buffer_exp_avg'] is None:
                        total_params = sum(m.numel() for m in all_exp_avg)
                        device = all_exp_avg[0].device
                        dtype = all_exp_avg[0].dtype
                        group_state['reuse_buffer_exp_avg'] = torch.zeros(total_params, device=device, dtype=dtype)
                        group_state['reuse_buffer_exp_avg_sq'] = torch.zeros(total_params, device=device, dtype=dtype)
                    
                    # Copy moment estimates into buffers (avoids repeated allocations)
                    offset = 0
                    reuse_buffer_exp_avg = group_state['reuse_buffer_exp_avg']
                    reuse_buffer_exp_avg_sq = group_state['reuse_buffer_exp_avg_sq']
                    
                    for m, v in zip(all_exp_avg, all_exp_avg_sq):
                        numel = m.numel()
                        reuse_buffer_exp_avg[offset:offset+numel].copy_(m)
                        reuse_buffer_exp_avg_sq[offset:offset+numel].copy_(v)
                        offset += numel
                    
                    # Compute AGAR on concatenated moments
                    agar = self._compute_agar(
                        reuse_buffer_exp_avg[:offset],
                        reuse_buffer_exp_avg_sq[:offset],
                        eps,
                        agar_clamp_factor
                    )
                    
                    agar_value = agar.item()
                    group_state['current_agar'] = agar_value
                    
                    # Tau calibration and adaptation
                    if not group_state['tau_initialized']:
                        group_state['agar_buffer'].append(agar_value)
                        
                        # Diagnostic logging during calibration
                        if self._step_count % 10 == 0 and len(group_state['agar_buffer']) > 0:
                            agars = list(group_state['agar_buffer'])
                            self._log(3, f"Step {self._step_count} - AGAR: min={min(agars):.4f}, median={np.median(agars):.4f}, max={max(agars):.4f}")
                        
                        if len(group_state['agar_buffer']) >= tau_init_steps:
                            tau = self._calibrate_tau(group_state['agar_buffer'], tau_clip_range, group_idx)
                            group_state['tau_adapter'].tau = tau
                            group_state['tau_adapter'].tau_calibrated = tau  # Anchor dead zone to calibrated value
                            group_state['tau_initialized'] = True
                            group_state['agar_buffer'].clear()
                            self._log(2, f"Group {group_idx}: Tau calibrated to {tau:.4f} from {tau_init_steps} samples")
                    else:
                        # Post-calibration: adapt tau using drift-detecting EMA
                        group_state['tau_adapter'].update(agar_value)
                    
                    # Universal sigmoid-based confidence mapping
                    if group_state['tau_initialized']:
                        mu = group_state.get('agar_mean', agar_value)
                        sigma = group_state.get('agar_std', 0.1)
                        c_min_adaptive = group_state.get('c_min', c_min)
                        
                        # Sigmoid mapping: confidence = c_min + (1 - c_min) * sigmoid((agar - μ) / σ)
                        # This naturally adapts to any distribution:
                        # - High μ, low σ (clean): Most samples get high confidence
                        # - Low μ, low σ (pervasive noise): Confidence scales smoothly from c_min
                        # - High σ (mixed): Strong discrimination between low/high AGAR
                        z_score = (agar_value - mu) / sigma
                        sigmoid = 1.0 / (1.0 + np.exp(-z_score))
                        confidence_value = c_min_adaptive + (1.0 - c_min_adaptive) * sigmoid
                        
                        confidence_value = float(np.clip(confidence_value, c_min_adaptive, 1.0))
                    else:
                        # Pre-calibration: simple passthrough
                        confidence_value = float(np.clip(agar_value, c_min, 1.0))
                    
                    group_state['current_confidence'] = confidence_value
                    
                    # Diagnostic logging
                    if group_state['tau_initialized'] and self._step_count % 100 == 0:
                        mu = group_state.get('agar_mean', 0)
                        sigma = group_state.get('agar_std', 0)
                        self._log(3, f"Step {self._step_count} - AGAR={agar_value:.4f}, μ={mu:.4f}, "
                                     f"σ={sigma:.4f}, Confidence={confidence_value:.4f}")
                    
                    confidence_tensor = torch.tensor(confidence_value, device=all_exp_avg[0].device, dtype=all_exp_avg[0].dtype)
                else:
                    confidence_tensor = torch.tensor(c_min)
                
                # Apply parameter updates with confidence-scaled learning rate
                for p in group['params']:
                    if p.grad is None:
                        continue
                    
                    self._apply_weight_update(p, self.state[p], lr, weight_decay, 
                                             eps, confidence_tensor, beta1, beta2)
            
            # Per-parameter AGAR mode: compute separately for each parameter
            else:
                for p in group['params']:
                    if p.grad is None:
                        continue
                    
                    self._validate_gradient(p.grad, group_idx)
                    state = self._init_param_state(p)
                    self._update_moments(state, p.grad, beta1, beta2)
                    
                    # Compute per-parameter AGAR
                    agar = self._compute_agar(state['exp_avg'], state['exp_avg_sq'], eps, agar_clamp_factor)
                    
                    agar_value = agar.item()
                    
                    # Store first parameter's AGAR as representative for monitoring
                    if group_state['current_agar'] is None:
                        group_state['current_agar'] = agar_value
                    
                    # Tau calibration and adaptation
                    if not group_state['tau_initialized']:
                        group_state['agar_buffer'].append(agar_value)
                        
                        # Diagnostic logging during calibration
                        if self._step_count % 10 == 0 and len(group_state['agar_buffer']) > 0:
                            agars = list(group_state['agar_buffer'])
                            self._log(3, f"Step {self._step_count} - AGAR: min={min(agars):.4f}, median={np.median(agars):.4f}, max={max(agars):.4f}")
                        
                        if len(group_state['agar_buffer']) >= tau_init_steps:
                            tau = self._calibrate_tau(group_state['agar_buffer'], tau_clip_range, group_idx)
                            group_state['tau_adapter'].tau = tau
                            group_state['tau_adapter'].tau_calibrated = tau  # Anchor dead zone to calibrated value
                            group_state['tau_initialized'] = True
                            group_state['agar_buffer'].clear()
                            self._log(2, f"Group {group_idx}: Tau calibrated to {tau:.4f} from {tau_init_steps} samples")
                    else:
                        # Post-calibration: adapt tau using drift-detecting EMA
                        group_state['tau_adapter'].update(agar_value)
                    
                    # Universal sigmoid-based confidence mapping
                    if group_state['tau_initialized']:
                        mu = group_state.get('agar_mean', agar_value)
                        sigma = group_state.get('agar_std', 0.1)
                        c_min_adaptive = group_state.get('c_min', c_min)
                        
                        # Sigmoid mapping: confidence = c_min + (1 - c_min) * sigmoid((agar - μ) / σ)
                        z_score = (agar_value - mu) / sigma
                        sigmoid = 1.0 / (1.0 + np.exp(-z_score))
                        confidence_value = c_min_adaptive + (1.0 - c_min_adaptive) * sigmoid
                        
                        confidence_value = float(np.clip(confidence_value, c_min_adaptive, 1.0))
                    else:
                        # Pre-calibration: simple passthrough
                        confidence_value = float(np.clip(agar_value, c_min, 1.0))
                    
                    confidence_tensor = torch.tensor(confidence_value, device=p.device, dtype=p.dtype)
                    
                    # Store first parameter's confidence as representative for monitoring
                    if group_state['current_confidence'] is None:
                        group_state['current_confidence'] = confidence_value
                    
                    # Diagnostic logging
                    if group_state['tau_initialized'] and self._step_count % 100 == 0:
                        mu = group_state.get('agar_mean', 0)
                        sigma = group_state.get('agar_std', 0)
                        self._log(3, f"Step {self._step_count} - AGAR={agar_value:.4f}, μ={mu:.4f}, "
                                     f"σ={sigma:.4f}, Confidence={confidence_value:.4f}")
                    
                    # Apply parameter update
                    self._apply_weight_update(p, state, lr, weight_decay, 
                                             eps, confidence_tensor, beta1, beta2)
        
        return loss
    
    def state_dict(self):
        """
        Return the optimizer state as a dictionary.
        
        Includes all parameter states, hyperparameters, and internal calibration data.
        Compatible with torch.save() for checkpointing.
        
        Returns:
            dict: Complete optimizer state including:
                - Parameter-specific states (exp_avg, exp_avg_sq, step)
                - Group-level calibration data (tau, agar_buffer)
                - Global step counter
        """
        state_dict = super().state_dict()
        
        # Serialize group states (convert deque to list)
        serializable_group_states = {}
        for idx, gs in self._group_states.items():
            serializable_group_states[idx] = {
                'tau_initialized': gs['tau_initialized'],
                'agar_buffer': list(gs['agar_buffer']),
                'agar_buffer_maxlen': gs['agar_buffer'].maxlen,
                'adapter_tau': gs['tau_adapter'].tau,
                'adapter_tau_calibrated': gs['tau_adapter'].tau_calibrated,
                'adapter_mean': gs['tau_adapter'].mean_agar,
                'adapter_var': gs['tau_adapter'].ema_var,
                'agar_mean': gs.get('agar_mean'),
                'agar_std': gs.get('agar_std'),
                'agar_median': gs.get('agar_median'),
                'agar_p10': gs.get('agar_p10'),
                'agar_p90': gs.get('agar_p90'),
                'c_min': gs.get('c_min'),
            }
        
        state_dict['_group_states'] = serializable_group_states
        state_dict['_step_count'] = self._step_count
        
        return state_dict
    
    def load_state_dict(self, state_dict):
        """
        Load optimizer state from a dictionary.
        
        Restores all parameter states, hyperparameters, and internal calibration data.
        Compatible with torch.load() for checkpoint restoration.
        
        Args:
            state_dict (dict): Optimizer state dictionary (typically from state_dict())
        
        Note:
            Automatically handles conversion of serialized lists back to deque objects
            for AGAR buffer management.
        """
        # Restore group states (convert list back to deque)
        if '_group_states' in state_dict:
            loaded_states = state_dict.pop('_group_states')
            # Ensure keys are integers (they may be strings after JSON serialization)
            self._group_states = {}
            for idx, gs in loaded_states.items():
                idx_int = int(idx) if isinstance(idx, str) else idx
                
                maxlen = gs.pop('agar_buffer_maxlen', None)
                buffer_list = gs.pop('agar_buffer', [])
                gs['agar_buffer'] = deque(buffer_list, maxlen=maxlen)
                
                # Restore adapter state
                tau_clip_range = self.param_groups[idx_int]['tau_clip_range']
                tau_dead_zone = self.param_groups[idx_int]['tau_dead_zone']
                adapter = DDEAdapter(1.0, tau_clip_range, dead_zone_factor=tau_dead_zone)
                adapter.tau = gs.pop('adapter_tau', 1.0)
                adapter.tau_calibrated = gs.pop('adapter_tau_calibrated', None)
                adapter.mean_agar = gs.pop('adapter_mean', 1.0)
                adapter.ema_var = gs.pop('adapter_var', 0.01)
                gs['tau_adapter'] = adapter
                
                # Initialize missing fields
                gs.setdefault('reuse_buffer_exp_avg', None)
                gs.setdefault('reuse_buffer_exp_avg_sq', None)
                gs.setdefault('current_agar', None)
                gs.setdefault('current_confidence', None)
                gs.setdefault('agar_mean', None)
                gs.setdefault('agar_std', None)
                gs.setdefault('agar_median', None)
                gs.setdefault('agar_p10', None)
                gs.setdefault('agar_p90', None)
                gs.setdefault('c_min', self.param_groups[idx_int].get('c_min', 0.1))
                
                self._group_states[idx_int] = gs
        
        # Restore step count
        if '_step_count' in state_dict:
            self._step_count = state_dict.pop('_step_count')
        
        super().load_state_dict(state_dict)

In [ ]:
"""
Benchmark B8: Noisy Instruction Following (Phi-2 + QLoRA)

Tests CASMO's ability to filter out noisy instruction data (misaligned preferences) 
during LLM fine-tuning.

Model: Microsoft Phi-2 (2.7B)
Technique: QLoRA (4-bit)
Dataset: UltraFeedback (30% Noise Injected)
"""

import sys
import os
import argparse
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from torch.utils.data import DataLoader

# -----------------------------------------------------------------------------
# Setup
# -----------------------------------------------------------------------------

def get_model_and_tokenizer(model_name="microsoft/phi-2"):
    print(f"Loading {model_name} with 4-bit quantization...")
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
    
    model = prepare_model_for_kbit_training(model)
    
    # LoRA Config
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["Wqkv", "out_proj", "fc1", "fc2"], # Phi-2 specific modules
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    
    return model, tokenizer

# -----------------------------------------------------------------------------
# Training
# -----------------------------------------------------------------------------

def run_training(optimizer_name, epochs=1, batch_size=8, grad_accum_steps=4, lr=2e-4, noise_ratio=0.3):
    print(f"\n{'='*60}")
    print(f"Running: {optimizer_name.upper()} | Noise: {noise_ratio*100}%")
    print(f"Batch: {batch_size} | Accum: {grad_accum_steps} | Effective Batch: {batch_size*grad_accum_steps}")
    print(f"{'='*60}\n")
    
    model, tokenizer = get_model_and_tokenizer()
    
    # Dataset
    train_dataset = NoisyInstructDataset(tokenizer, split='train', noise_ratio=noise_ratio)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Optimizer
    if optimizer_name == 'casmo':
        optimizer = CASMO(
            model.parameters(),
            lr=lr,
            tau_init_steps=100, # Fast calibration for short benchmark
            weight_decay=0.01,
            granularity='group', # 'group' is recommended for large models
            betas=(0.9, 0.999),
            log_level=1
        )
    else:
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
        
    results = {
        'losses': [],
        'noisy_confidences': [],
        'clean_confidences': [],
        'steps': []
    }
    
    step = 0
    global_step = 0
    model.train()
    optimizer.zero_grad()
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        progress_bar = tqdm(train_loader, desc="Training")
        
        current_batch_noisy = []
        
        for i, batch in enumerate(progress_bar):
            input_ids = batch['input_ids'].to(model.device)
            attention_mask = batch['attention_mask'].to(model.device)
            labels = batch['labels'].to(model.device)
            is_noisy = batch['is_noisy'] # [Batch]
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss / grad_accum_steps # Scale loss
            
            loss.backward()
            
            # Track noise for the accumulated batch
            current_batch_noisy.append(is_noisy.float().mean().item())
            
            if (i + 1) % grad_accum_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                
                # Log Metrics (once per effective step)
                current_loss = loss.item() * grad_accum_steps
                results['losses'].append(current_loss)
                results['steps'].append(global_step)
                
                # CASMO Confidence Tracking
                if optimizer_name == 'casmo':
                    # Get average confidence across all groups
                    confs = []
                    for g_id, state in optimizer._group_states.items():
                        if 'current_confidence' in state:
                            confs.append(state['current_confidence'])
                    
                    avg_conf = np.mean(confs) if confs else 1.0
                    
                    # Heuristic: If >50% of the *accumulated* batch is noisy
                    avg_noise_ratio = np.mean(current_batch_noisy)
                    if avg_noise_ratio > 0.5:
                        results['noisy_confidences'].append(avg_conf)
                    else:
                        results['clean_confidences'].append(avg_conf)
                
                current_batch_noisy = [] # Reset
                progress_bar.set_postfix({'loss': f"{current_loss:.4f}"})
                global_step += 1
                
                if global_step >= 150: # Run enough steps to see post-calibration behavior
                    print("Step limit reached (150). Stopping.")
                    break
        
        if global_step >= 150:
            break
            
    return results

# -----------------------------------------------------------------------------
# Main
# -----------------------------------------------------------------------------

def main():    
    # Run CASMO
    casmo_res = run_training('casmo', epochs=1, batch_size=8, noise_ratio=0.3)
    
    # Run AdamW
    adam_res = run_training('adamw', epochs=1, batch_size=8, noise_ratio=0.3)
    
    # Plot
    results_dir = 'results'
    os.makedirs(results_dir, exist_ok=True)
    
    plt.figure(figsize=(12, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(casmo_res['losses'], label='CASMO', alpha=0.7)
    plt.plot(adam_res['losses'], label='AdamW', alpha=0.7)
    plt.title('Training Loss')
    plt.xlabel('Step')
    plt.legend()
    
    # Confidence
    plt.subplot(1, 2, 2)
    if casmo_res['noisy_confidences']:
        plt.hist(casmo_res['clean_confidences'], alpha=0.5, label='Clean Batches', bins=20, color='green')
        plt.hist(casmo_res['noisy_confidences'], alpha=0.5, label='Noisy Batches', bins=20, color='red')
        plt.title('CASMO Confidence Distribution')
        plt.xlabel('Confidence')
        plt.legend()
        
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'instruct_results.png'))
    print(f"Results saved to {results_dir}")

if __name__ == '__main__':
    main()


Running: CASMO | Noise: 30.0%
Batch: 8 | Accum: 4 | Effective Batch: 32

Loading microsoft/phi-2 with 4-bit quantization...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Processing 10000 samples with 30.0% noise...
Epoch 1/1


Training:   0%|          | 0/1250 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Training:  46%|████▋     | 580/1250 [1:20:26<1:35:48,  8.58s/it, loss=1.2656]